In [1]:
# Import dependencies and libraries
import gmaps
import pandas as pd
import os
import requests

from citipy import citipy

# Google developer API key
from config import api_key

# Configure gmaps
gmaps.configure(api_key=api_key)


In [2]:
# Importing csv with cities data

## Retrieving file list
avail_files = []
path = os.path.join('..', 'WeatherPy', 'results_csv')
for _ in os.scandir(path):
    if os.path.isfile:
        (avail_files.append(_.name))
    
# Saving the files list to a dataframe to allow user input by number
avail_files = pd.DataFrame({'Files Available': avail_files})
print (avail_files)

# For testing purposes, switch comment after testing
import_file = int (input("Type the index of the file you want to use: "))
#import_file = 2

# Opening csv file and converting to dataframe
csv_path = os.path.join('..', 'WeatherPy', 'results_csv', avail_files.loc[import_file, 'Files Available'])
weather_data = pd.read_csv(csv_path, index_col=0)

weather_data

               Files Available
0  weather_data_1619218389.csv
1  weather_data_1619456878.csv
2  weather_data_1619635358.csv
3        weather_data_test.csv
Type the index of the file you want to use: 2


,Cities,Latitude,Longitude,Temperature (°F),Humidity,Cloudiness,Wind Speed (mph)
0,mancio lima,-7.6142,-72.8958,84.20,74,75,3.44
1,coari,-4.0850,-63.1414,77.04,95,98,3.87
2,berga,60.3920,5.3280,46.87,40,20,3.44
3,saldanha,-33.0117,17.9442,59.00,72,9,12.66
4,mihijam,23.8526,86.8788,88.00,34,1,7.72
...,...,...,...,...,...,...,...
593,coquimbo,-29.9533,-71.3436,64.40,68,0,11.50
594,jiwani,25.0500,61.7417,83.26,72,92,14.81
595,garmsar,35.2182,52.3409,76.98,30,100,6.26
596,praia da vitoria,38.7333,-27.0667,60.46,82,75,12.12


In [3]:
# Obtaining the humidity level heat map

# Centering the map on point 0 and fixing the zoom level
fig = gmaps.figure(center = (0,0), zoom_level=2)

# Getting lat/long to plot
cities = weather_data[['Latitude', 'Longitude']]

# Creating the heat layer with humidity levels
humidity = weather_data['Humidity']
heat_layer = gmaps.heatmap_layer(cities, weights=humidity, dissipating=False, point_radius=3)
fig.add_layer(heat_layer)

# Displaying the map
fig


Figure(layout=FigureLayout(height='420px'))

In [4]:
# Narrowing down the dataframe to ideal conditions

# Filter parameters as variables for easily changing them if needed
max_temp = 80
min_temp = 70
wind_speed = 10
cloudiness = 0

# Filtering the dataframe
ideal_cities = weather_data.copy(deep=True).reset_index(drop=True)
ideal_cities = ideal_cities.loc[(ideal_cities['Temperature (°F)'] > min_temp) &
                                (ideal_cities['Temperature (°F)'] < max_temp) &
                                (ideal_cities['Wind Speed (mph)'] < wind_speed) &
                                (ideal_cities['Cloudiness'] == cloudiness),:]


# Creating the new map

# Centering the map on point 0 and fixing the zoom level
humid_map = gmaps.figure(center = (0,0), zoom_level=2)

# Getting lat/long to plot
cities = ideal_cities[['Latitude', 'Longitude']]

# Creating the heat layer with humidity levels
humidity = ideal_cities['Humidity']
heat_layer = gmaps.heatmap_layer(cities, weights=humidity, dissipating=False, point_radius=3)
humid_map.add_layer(heat_layer)

# Displaying the map
humid_map

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# Variable initialization
hotel_name = []

# Creating a sample of 10 cities to avoid excesive API calls
if len(ideal_cities.index) < 19:
    hotel_list = ideal_cities
else:
    hotel_list = ideal_cities.sample(20)

# Search parameters
search = 'Hotel'
search_radius = 5000
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# For loop to create the parameters dictionary and make the API calls
for hotel in hotel_list['Cities']:
    
    # Retrieving lat/lon from the dataframe
    latitude = hotel_list['Latitude'].loc[hotel_list['Cities'] == hotel].tolist()
    longitude = hotel_list['Longitude'].loc[hotel_list['Cities'] == hotel].tolist()
    coordinates = str(latitude[0]) + ', ' + str(longitude[0])
    
    # Dynamic search parameters
    parameters = {'location': coordinates,
                 'keyword': search,
                 'radius': search_radius,
                 'key': api_key}
    
    #API call and response
    response = requests.get(base_url, params=parameters)
    hotel_data = response.json()
       
    #Saving the hotel name to a list
    try:
        hotel_name.append(hotel_data['results'][0]['name'])
        print(f"Processing {hotel}, found {hotel_data['results'][0]['name']}")
    except (IndexError):
        hotel_name.append('None found')
        print(f'Processing {hotel}, none found')
        

Processing posadas, found City Hotel
Processing paysandu, found Hotel Casagrande
Processing murgab, none found
Processing abu kamal, none found
Processing concepcion del uruguay, found Posta del Torreon Boutique
Processing hurghada, found Albatros White Beach - All Inclusive
Processing monte patria, found Hostal korona
Processing santa fe, found InterTower Hotel
Processing villa maria, found Amerian Villa María Park Hotel
Processing ovalle, found Hotel Limari
Processing rivera, found Hotel Casino & Resort Rivera
Processing santiago del estero, found Hotel Altos del Estero
Processing cordoba, found Azur Real Hotel Boutique
Processing rengo, found Hospedaje Alborada
Processing tacuarembo, found Hotel y Museo La Cumparsita
Processing chiredzi, found The Nesbitt Arms
Processing najran, none found
Processing kifri, none found
Processing neuquen, found Hotel Huemul
Processing san lorenzo, found La Misión Hotel Boutique


In [7]:
# Appending hotels found to dataframe
hotel_list['Hotel'] = hotel_name

# Obtaining country
country_list = []
for city in hotel_list['Cities']:
    
    # Retrieving lat/lon from the dataframe
    latitude = hotel_list['Latitude'].loc[hotel_list['Cities'] == city].tolist()
    longitude = hotel_list['Longitude'].loc[hotel_list['Cities'] == city].tolist()
    citi = citipy.nearest_city(latitude[0], longitude[0])
    country = citi.country_code
        
    # Appending country code
    country_list.append(country)
    
# Adding country column to dataframe    
hotel_list['Country'] = country_list
hotel_list = hotel_list.reset_index(drop=True)
hotel_list


,Cities,Latitude,Longitude,Temperature (°F),Humidity,Cloudiness,Wind Speed (mph),Hotel,Country
0,posadas,-27.3671,-55.8961,72.50,43,0,3.44,City Hotel,ar
1,paysandu,-32.3214,-58.0756,75.99,48,0,1.70,Hotel Casagrande,uy
2,murgab,37.4966,61.9714,78.80,41,0,5.57,None found,tm
3,abu kamal,34.4506,40.9171,79.36,20,0,6.51,None found,sy
4,concepcion del uruguay,-32.4825,-58.2372,75.00,49,0,1.01,Posta del Torreon Boutique,ar
5,hurghada,27.2574,33.8129,76.01,40,0,8.84,Albatros White Beach - All Inclusive,eg
6,monte patria,-30.6919,-70.9467,79.05,34,0,6.04,Hostal korona,cl
7,santa fe,-31.6333,-60.7000,75.67,41,0,4.00,InterTower Hotel,ar
8,villa maria,-32.4075,-63.2402,75.99,55,0,1.01,Amerian Villa María Park Hotel,ar
9,ovalle,-30.5983,-71.2003,78.22,40,0,5.26,Hotel Limari,cl


In [8]:
# Building the google map place marker

# Lists initialization
latitude = []
longitude = []
marker_labels = []
coordinates = []

# Dropping the rows for which the app didn't found an hotel nearby
hotel_list = hotel_list.loc[hotel_list['Hotel'] != 'None found', :].reset_index(drop=True)

# For loop to create information for all markers
for index, row in hotel_list.iterrows():
    coordinates.append((row['Latitude'], row['Longitude']))
    marker_labels.append('<p>'+ 
                         '<b>Hotel name</b>:<br />'+ row['Hotel'] + '<br />'+
                         '<b>City:</b><br />' + row['Cities'] + '<br />'+
                         '<b>Country:</b><br />' + row['Country'])

markers = gmaps.marker_layer(coordinates, info_box_content=marker_labels)
humid_map.add_layer(markers)
humid_map




Figure(layout=FigureLayout(height='420px'))